In [17]:
import requests
import sys
import pandas as pd
import numpy as np
from scipy import sparse
from io import StringIO
from matplotlib import pyplot as plt
import seaborn as sns
import itertools
import networkx as nx

In [18]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [19]:
df = pd.read_csv('/content/drive/MyDrive/NS_project/erasmus_mobility/mobilities_2018.csv', sep=';',low_memory=False, header=0)
for i, name in enumerate(df.columns):
    print(i, name)

0 Project Reference
1 Academic Year
2 Mobility Start Month
3 Mobility End Month
4 Mobility Duration
5 Activity (mob)
6 Field of Education
7 Participant Nationality
8 Education Level
9 Participant Gender
10 Participant Profile
11 Special Needs
12 Fewer Opportunities
13 GroupLeader
14 Participant Age
15 Sending Country Code
16 Sending City
17 Sending Organization
18 Sending Organisation Erasmus Code
19 Receiving Country Code
20 Receiving City
21 Receiving Organization
22 Receiving Organisation Erasmus Code
23 Participants


In [20]:
pd.options.display.max_columns=21
display(df)

,Project Reference,Academic Year,Mobility Start Month,Mobility End Month,Mobility Duration,Activity (mob),Field of Education,Participant Nationality,Education Level,Participant Gender,...,Participant Age,Sending Country Code,Sending City,Sending Organization,Sending Organisation Erasmus Code,Receiving Country Code,Receiving City,Receiving Organization,Receiving Organisation Erasmus Code,Participants
0,2015-3-PT02-KA105-002866,2018-2019,2018-01,2018-05,146.0,European Voluntary Service - Programme Countries,? Unknown ?,PT,??? - ? Unknown ?,Male,...,22,PT,VISEU,ADAMASTOR - ASSOCIACAO CULTURAL,-,PL,Poznań,Stowarzyszenie Jeden Swiat,-,1
1,2015-3-PT02-KA105-002866,2018-2019,2018-01,2018-05,146.0,European Voluntary Service - Programme Countries,? Unknown ?,PT,??? - ? Unknown ?,Male,...,26,PT,VISEU,ADAMASTOR - ASSOCIACAO CULTURAL,-,PL,Poznań,Stowarzyszenie Jeden Swiat,-,1
2,2016-1-AT01-KA101-016290,2018-2019,2018-01,2018-01,6.0,Structured Courses/Training Events,Teacher training without subject specialization,AT,??? - ? Unknown ?,Female,...,27,AT,Wien,Neue Mittelschule Glasergasse,-,SI,Portoro�,Osnovna sola Lucija,-,1
3,2016-1-AT01-KA101-016290,2018-2019,2018-01,2018-01,6.0,Structured Courses/Training Events,Teacher training without subject specialization,AT,??? - ? Unknown ?,Female,...,28,AT,Vienna,NMSI & JHS Konstanziagasse 50,-,SI,Portoro�,Osnovna sola Lucija,-,1
4,2016-1-AT01-KA101-016290,2018-2019,2018-01,2018-01,6.0,Structured Courses/Training Events,Teacher training without subject specialization,AT,??? - ? Unknown ?,Female,...,35,AT,Wien,GTKMS/GTNMS Anton-Sattler-Gasse,-,SI,Portoro�,Osnovna sola Lucija,-,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690326,2018-2-UK01-KA125-048625,2018-2019,2018-09,2019-07,319.0,Individual Volunteering Activities,? Unknown ?,ES,??? - ? Unknown ?,Female,...,21,ES,Madrid,Asociación para la integración y Progreso de l...,-,UK,Stackpole,The National Trust for Places of Historic Inte...,-,1
690327,2018-2-UK01-KA125-048626,2018-2019,2018-10,2019-03,137.0,Individual Volunteering Activities,? Unknown ?,ES,??? - ? Unknown ?,Male,...,27,ES,BARCELONA,COORDINADORA D'ORGANITZADORS DE CAMPS DE TREBA...,-,UK,LONDON,VOLUNTEER ACTION FOR PEACE,-,1
690328,2018-2-UK01-KA125-048626,2018-2019,2018-11,2019-04,170.0,Individual Volunteering Activities,? Unknown ?,CZ,??? - ? Unknown ?,Female,...,24,CZ,PRAHA,INEX - SDRUZENI DOBROVOLNYCH AKTIVIT Z S,-,UK,LONDON,VOLUNTEER ACTION FOR PEACE,-,1
690329,2018-2-UK01-KA125-048626,2018-2019,2018-11,2019-10,351.0,Individual Volunteering Activities,? Unknown ?,FR,??? - ? Unknown ?,Male,...,23,FR,PARIS,SOLIDARITES JEUNESSES MCP,-,UK,LONDON,VOLUNTEER ACTION FOR PEACE,-,1


In [21]:
mobility_df = df[df[df.columns[10]]=='Learner']
mobility_df = mobility_df[mobility_df['Activity (mob)'].str.contains('Student')]
mobility_df = mobility_df[~mobility_df['Activity (mob)'].str.contains('traineeship')]
mobility_df = mobility_df.loc[:, df.columns[np.array((5, 15, 17, 19, 21, 23))]]
print((mobility_df['Receiving Organization'].nunique()))
print((mobility_df['Sending Organization'].nunique()))

3789
4306


In [22]:
### UNI ITALIANE ###

# prendi solo le righe dove 'Receiving/Sending Organization' sono italiane
sending_uni_IT = df[df[df.columns[15]]=='IT']
receiving_uni_IT = df[df[df.columns[19]]=='IT']

column_list = ["Activity (mob)", "Sending Country Code", "Sending City", "Sending Organization", "Receiving Country Code", "Receiving City", "Receiving Organization", "Participants"]
sending_uni_IT_filtered = sending_uni_IT[column_list]
receiving_uni_IT_filtered = receiving_uni_IT[column_list]
sending_uni_IT_filtered.to_csv("sending_uni_IT_filtered.csv", index=False)
receiving_uni_IT_filtered.to_csv("receiving_uni_IT_filtered.csv", index=False)

In [23]:
### UNI ITALIANE ###

from_ita = {}
for row in sending_uni_IT_filtered["Receiving Country Code"]:
  for word in row.split(" "):
    #if the word is encountered for the first time add to dict as key and set its value to 0
    from_ita.setdefault(word,0)
    #increase the value (i.e the count) of the word by 1 every time it is encountered
    from_ita[word] += 1

from_ita = sorted(from_ita.items(), key=lambda x:x[1], reverse=True)
sortdict = dict(from_ita)
print("Number of students from italian uni gone in erasmus abroad: ")
abroad_df = pd.DataFrame.from_dict(sortdict, orient='index').reset_index()
abroad_df.rename(columns = {'index':'Country', 0:'Count'}, inplace=True)
abroad_df.sort_values(by=['Count'], ascending=False, inplace=True)
print(abroad_df)

to_ita = {}
for row in receiving_uni_IT_filtered["Sending Country Code"]:
  for word in row.split(" "):
    #if the word is encountered for the first time add to dict as key and set its value to 0
    to_ita.setdefault(word,0)
    #increase the value (i.e the count) of the word by 1 every time it is encountered
    to_ita[word] += 1

to_ita = sorted(to_ita.items(), key=lambda x:x[1], reverse=True)
sortdict2 = dict(to_ita)
print("Number of foreign students gone in erasmus in Italy: ")
incoming_df = pd.DataFrame.from_dict(sortdict2, orient='index').reset_index()
incoming_df.rename(columns = {'index':'Country', 0:'Count'}, inplace=True)
incoming_df.sort_values(by=['Count'], ascending=False, inplace=True)
print(incoming_df)

Number of students from italian uni gone in erasmus abroad: 
   Country  Count
0       ES  16140
1       UK   6489
2       FR   6043
3       DE   5950
4       PT   3302
..     ...    ...
85      SN      1
86      SO      1
87      BD      1
88      SV      1
89      KZ      1

[90 rows x 2 columns]
Number of foreign students gone in erasmus in Italy: 
   Country  Count
0       ES  14719
1       FR   5721
2       DE   5413
3       PL   3690
4       TR   2943
..     ...    ...
91      LA      2
92      MG      2
93      NZ      2
94      GH      1
95      AO      1

[96 rows x 2 columns]


In [24]:
uniIT_tx = {}
for row in sending_uni_IT_filtered['Sending Organization']:
  if(row.__contains__("UNIVERSITA") or row.__contains__("UNIVERSITA'") or row.__contains__("POLITECNICO")):
    uniIT_tx.setdefault(row,0)
    uniIT_tx[row] += 1
uniIT_tx = sorted(uniIT_tx.items(), key=lambda x:x[1], reverse=True)
sortdict_tx = dict(uniIT_tx)
print("Number of students from italian uni gone in erasmus abroad: ")
uniIT_tx_df = pd.DataFrame.from_dict(sortdict_tx, orient='index').reset_index()
uniIT_tx_df.rename(columns = {'index':'Sending_Uni', 0:'Count'}, inplace=True)
uniIT_tx_df.sort_values(by=['Count'], ascending=False, inplace=True)
uniIT_tx_df = uniIT_tx_df[uniIT_tx_df.Count > 10] # keep row only if count > 10
print(uniIT_tx_df)

uniIT_rx = {}
for row in receiving_uni_IT_filtered['Receiving Organization']:
  if(row.__contains__("UNIVERSITA") or row.__contains__("UNIVERSITA'") or row.__contains__("POLITECNICO")):
    uniIT_rx.setdefault(row,0)
    uniIT_rx[row] += 1
uniIT_rx = sorted(uniIT_rx.items(), key=lambda x:x[1], reverse=True)
sortdict_rx = dict(uniIT_rx)
print("Number of students from abroad that have chosen Italian uni for erasmus: ")
uniIT_rx_df = pd.DataFrame.from_dict(sortdict_rx, orient='index').reset_index()
uniIT_rx_df.rename(columns = {'index':'Receiving_Uni', 0:'Count'}, inplace=True)
uniIT_rx_df.sort_values(by=['Count'], ascending=False, inplace=True)
uniIT_rx_df = uniIT_rx_df[uniIT_rx_df.Count > 10] # keep row only if count > 10
print(uniIT_rx_df)

Number of students from italian uni gone in erasmus abroad: 
                                          Sending_Uni  Count
0        ALMA MATER STUDIORUM - UNIVERSITA DI BOLOGNA   2860
1                    UNIVERSITA DEGLI STUDI DI PADOVA   2059
2          UNIVERSITA DEGLI STUDI DI ROMA LA SAPIENZA   1887
3                    UNIVERSITA DEGLI STUDI DI TORINO   1672
4                               POLITECNICO DI MILANO   1240
..                                                ...    ...
78          UNIVERSITA' PER STRANIERI DANTE ALIGHIERI     19
80  UNIVERSITA DEGLI STUDI GUGLIELMO MARCONI - TEL...     18
79   LIBERA UNIVERSITA DI LINGUE E COMUNICAZIONE IULM     18
81  SCUOLA SUPERIORE DI STUDI UNIVERSITARI E DI PE...     17
82  SCUOLA SUPERIORE DI STUDI UNIVERSITARI E DI PE...     16

[83 rows x 2 columns]
Number of students from abroad that have chosen Italian uni for erasmus: 
                                        Receiving_Uni  Count
0        ALMA MATER STUDIORUM - UNIVERSITA DI BOL

In [25]:
# relazione tra numero di studenti erasmus e 1. ranking QS 
#                                            2. ranking ARWU 
#                                            3. numero totale di studenti dell'università

rank_rx = pd.read_csv('/content/drive/MyDrive/NS_project/erasmus_mobility/ranking_receiving_uni_IT.csv', sep=';', low_memory=False, header=0)
rank_tx = pd.read_csv('/content/drive/MyDrive/NS_project/erasmus_mobility/ranking_sending_uni_IT.csv', sep=';', low_memory=False, header=0)

In [26]:
rank_rx

,Universities,Total incoming students,QS ranking 2019 (world),QS ranking 2019 (Italy),ARWU ranking 2019 (world),ARWU ranking 2019 (Italy),Total number of students,% incoming students
0,ALMA MATER STUDIORUM - UNIVERSITA DI BOLOGNA,2633,180,4.0,201-300,5.0,81183.0,"3,24%"
1,UNIVERSITA DEGLI STUDI DI ROMA LA SAPIENZA,1764,217,5.0,151-200,1.0,100362.0,"1,76%"
2,UNIVERSITA DEGLI STUDI DI PADOVA,1600,249,6.0,201-300,7.0,59337.0,"2,70%"
3,POLITECNICO DI MILANO,1414,156,1.0,201-300,4.0,45544.0,"3,10%"
4,UNIVERSITA DEGLI STUDI DI FIRENZE,1258,501-510,13.0,201-300,6.0,52569.0,"2,39%"
...,...,...,...,...,...,...,...,...
69,UNIVERSITA VITA-SALUTE SAN RAFFAELE,37,NaN,NaN,501-600,24.0,2740.0,"1,35%"
70,UNIVERSITA DEGLI STUDI MAGNA GRAECIA DI CATANZARO,26,NaN,NaN,NaN,NaN,10828.0,"0,24%"
71,SCUOLA SUPERIORE DI STUDI UNIVERSITARI E DI PE...,25,167,2.0,NaN,NaN,NaN,NaN
72,UNIVERSITA DELLA VALLE D'AOSTA,22,NaN,NaN,NaN,NaN,1071.0,"2,05%"


In [27]:
rank_tx

,Universities,Total outgoing students,QS ranking 2019 (world),QS ranking 2019 (Italy),ARWU ranking 2019 (world),ARWU ranking 2019 (Italy),Total number of students,% outgoing students
0,ALMA MATER STUDIORUM - UNIVERSITA DI BOLOGNA,2946,180,4.0,201-300,5.0,81183.0,"3,63%"
1,UNIVERSITA DEGLI STUDI DI PADOVA,2109,249,6.0,201-300,7.0,59337.0,"3,55%"
2,UNIVERSITA DEGLI STUDI DI ROMA LA SAPIENZA,1933,217,5.0,151-200,1.0,100362.0,"1,93%"
3,UNIVERSITA DEGLI STUDI DI TORINO,1732,571-580,15.0,201-300,8.0,72468.0,"2,39%"
4,UNIVERSITA DEGLI STUDI DI NAPOLI FEDERICO II,1321,472,11.0,301-400,9.0,76384.0,"1,73%"
...,...,...,...,...,...,...,...,...
71,UNIVERSITA VITA-SALUTE SAN RAFFAELE,49,NaN,NaN,501-600,24.0,2740.0,"1,79%"
72,SCUOLA SUPERIORE DI STUDI UNIVERSITARI E DI PE...,33,167,2.0,NaN,NaN,NaN,NaN
73,SCUOLA NORMALE SUPERIORE,31,175,3.0,401-500,12.0,576.0,"5,38%"
74,UNIVERSITA DEGLI STUDI GUGLIELMO MARCONI - TEL...,18,NaN,NaN,NaN,NaN,9521.0,"0,19%"
